In [1]:
import pandas as pd
import numpy as np
import os
import datetime as dt
import zipfile
import tempfile # for temporary file storage to avoid storing large datasets on github

In [2]:
base_dir = os.path.dirname(os.path.abspath("..\data"))
path = os.path.join(base_dir, r"data")
os.chdir(path)

In [3]:
# os.listdir()

#### Functions

In [4]:
# read zips from repo, extract data into temp dir, create dataframes
def zip_to_df(file_name):
    # create temporary directory  
    with tempfile.TemporaryDirectory() as tmpdirname:
        # collect data from zip file 
        with zipfile.ZipFile(file_name+".zip") as zip_file:
            file_list = zip_file.namelist()
            for file in file_list:
               zip_file.extract(file, tmpdirname)
            file_path = tmpdirname+"\\"+file_name
            # create and return dataframe
            df = pd.read_csv(file_path)
            return df

#### Total Demand

In [5]:
file_name = "totaldemand_nsw.csv"
totalDemand = zip_to_df(file_name)
totalDemand.head()

DATETIME REGIONID  TOTALDEMAND
0  2010-01-01 00:00:00     NSW1      7997.41
1  2010-01-01 00:05:00     NSW1      7946.51
2  2010-01-01 00:10:00     NSW1      7921.55
3  2010-01-01 00:15:00     NSW1      7837.84
4  2010-01-01 00:20:00     NSW1      7781.11

In [6]:
#update datatypes
totalDemand['DATETIME'] = pd.to_datetime(totalDemand['DATETIME'])
totalDemand.dtypes

DATETIME       datetime64[ns]
REGIONID               object
TOTALDEMAND           float64
dtype: object

#### Temperature

In [7]:
file_name = "temperature_nsw.csv"
temperature = zip_to_df(file_name)
temperature.head()

DATETIME  LOCATION  TEMPERATURE
0  2010-01-01 00:00:00   94766.0         23.1
1  2010-01-01 00:01:00   94766.0         23.1
2  2010-01-01 00:30:00   94766.0         22.9
3  2010-01-01 00:50:00   94766.0         22.7
4  2010-01-01 01:00:00   94766.0         22.6

In [8]:
#update datatypes
temperature['DATETIME'] = pd.to_datetime(temperature['DATETIME'])
temperature.dtypes

DATETIME       datetime64[ns]
LOCATION              float64
TEMPERATURE           float64
dtype: object

#### Forecast Demand

In [9]:
file_name = "forecastdemand_nsw.csv"
forecastDemand = zip_to_df(file_name)
forecastDemand.head()

DATETIME REGIONID  FORECASTDEMAND  PREDISPATCHSEQNO  PERIODID  \
0  2010-01-01 00:00:00     NSW1         7832.04        2009123018        71   
1  2010-01-01 00:00:00     NSW1         7832.04        2009123019        70   
2  2010-01-01 00:00:00     NSW1         7832.03        2009123020        69   
3  2010-01-01 00:00:00     NSW1         7832.03        2009123021        68   
4  2010-01-01 00:00:00     NSW1         7830.96        2009123022        67   

           LASTCHANGED  
0  2009-12-30 12:31:49  
1  2009-12-30 13:01:43  
2  2009-12-30 13:31:36  
3  2009-12-30 14:01:44  
4  2009-12-30 14:31:35

In [10]:
#update datatypes
forecastDemand['DATETIME'] = pd.to_datetime(forecastDemand['DATETIME'])
forecastDemand['LASTCHANGED'] = pd.to_datetime(forecastDemand['LASTCHANGED'])
forecastDemand.dtypes

DATETIME            datetime64[ns]
REGIONID                    object
FORECASTDEMAND             float64
PREDISPATCHSEQNO             int64
PERIODID                     int64
LASTCHANGED         datetime64[ns]
dtype: object

#### Energy Prices and Demand

In [11]:
#merge all files in folder
wd = path + r"\Price and Demand - 2010-22"
files = os.listdir(wd)
prices = pd.DataFrame([])
file_count = 0
for file in files:
    if file.endswith(".csv"):
        file_df = pd.read_csv(wd + "\\" + file)
        prices = prices.append(file_df, ignore_index=True)
        file_count += 1
print("Merged", file_count, "files")
#remove additional headers 
prices = prices[prices["REGION"]!="REGION"]
prices.head()

Merged 153 files


REGION       SETTLEMENTDATE TOTALDEMAND    RRP PERIODTYPE
0   NSW1  2010/01/01 00:30:00     7809.31   21.5      TRADE
1   NSW1  2010/01/01 01:00:00     7483.69  20.98      TRADE
2   NSW1  2010/01/01 01:30:00     7117.23  20.06      TRADE
3   NSW1  2010/01/01 02:00:00     6812.03  17.93      TRADE
4   NSW1  2010/01/01 02:30:00     6544.33  17.43      TRADE

In [12]:
#update datatypes
prices['SETTLEMENTDATE'] = pd.to_datetime(prices['SETTLEMENTDATE'])
prices['TOTALDEMAND'] = prices['TOTALDEMAND'].astype(float)
prices['RRP'] = prices['RRP'].astype(float)
prices.dtypes

REGION                    object
SETTLEMENTDATE    datetime64[ns]
TOTALDEMAND              float64
RRP                      float64
PERIODTYPE                object
dtype: object

#### BOM

In [13]:
# timeframe restriction
year_start = 2010

In [14]:
file_path = path + r"\Solar Exposure - Bankstown Airport\IDCJAC0016_066137_1800_Data.csv"
bomSolar = pd.read_csv(file_path)
bomSolar = bomSolar[bomSolar.Year >= year_start]
bomSolar.head()

Product code  Bureau of Meteorology station number  Year  Month  Day  \
7305   IDCJAC0016                                 66137  2010      1    1   
7306   IDCJAC0016                                 66137  2010      1    2   
7307   IDCJAC0016                                 66137  2010      1    3   
7308   IDCJAC0016                                 66137  2010      1    4   
7309   IDCJAC0016                                 66137  2010      1    5   

      Daily global solar exposure (MJ/m*m)  
7305                                  14.6  
7306                                  18.4  
7307                                   7.3  
7308                                  14.4  
7309                                  28.1

In [15]:
bomSolar.dtypes

Product code                             object
Bureau of Meteorology station number      int64
Year                                      int64
Month                                     int64
Day                                       int64
Daily global solar exposure (MJ/m*m)    float64
dtype: object

In [16]:
file_path = path + r"\Temperature - Bankstown Airport\IDCJAC0010_066137_1800_Data.csv"
bomTemp = pd.read_csv(file_path)
bomTemp = bomTemp[bomTemp.Year >= year_start]
bomTemp.head()

Product code  Bureau of Meteorology station number  Year  Month  Day  \
15341   IDCJAC0010                                 66137  2010      1    1   
15342   IDCJAC0010                                 66137  2010      1    2   
15343   IDCJAC0010                                 66137  2010      1    3   
15344   IDCJAC0010                                 66137  2010      1    4   
15345   IDCJAC0010                                 66137  2010      1    5   

       Maximum temperature (Degree C)  \
15341                            29.6   
15342                            29.5   
15343                            21.0   
15344                            24.0   
15345                            30.4   

       Days of accumulation of maximum temperature Quality  
15341                                          1.0       Y  
15342                                          1.0       Y  
15343                                          1.0       Y  
15344                                          1.0       Y  
15345                                          1.0       Y

In [17]:
bomTemp.dtypes

Product code                                    object
Bureau of Meteorology station number             int64
Year                                             int64
Month                                            int64
Day                                              int64
Maximum temperature (Degree C)                 float64
Days of accumulation of maximum temperature    float64
Quality                                         object
dtype: object

In [18]:
file_path = path + r"\Rainfall - Bankstown\IDCJAC0009_066137_1800_Data.csv"
bomRain = pd.read_csv(file_path)
bomRain = bomRain[bomRain.Year >= year_start]
bomRain.head()

Product code  Bureau of Meteorology station number  Year  Month  Day  \
15341   IDCJAC0009                                 66137  2010      1    1   
15342   IDCJAC0009                                 66137  2010      1    2   
15343   IDCJAC0009                                 66137  2010      1    3   
15344   IDCJAC0009                                 66137  2010      1    4   
15345   IDCJAC0009                                 66137  2010      1    5   

       Rainfall amount (millimetres)  \
15341                            0.2   
15342                            0.0   
15343                           15.2   
15344                            0.2   
15345                            0.0   

       Period over which rainfall was measured (days) Quality  
15341                                             1.0       N  
15342                                             1.0       N  
15343                                             1.0       N  
15344                                             1.0       N  
15345                                             1.0       N

In [19]:
bomRain.dtypes

Product code                                       object
Bureau of Meteorology station number                int64
Year                                                int64
Month                                               int64
Day                                                 int64
Rainfall amount (millimetres)                     float64
Period over which rainfall was measured (days)    float64
Quality                                            object
dtype: object

#### Calculated Columns

In [20]:
def collect_date_time(df, col):
    df["DATE"] = df[col].dt.date
    df["YEAR"] = df[col].dt.year
    df["MONTH"] = df[col].dt.month
    df["DAY"] = df[col].dt.day
    df["TIME"] = df[col].dt.time
    df["HOURS"] = df[col].dt.hour
    df["MINUTES"] = df[col].dt.minute
    return df

In [21]:
forecastDemand = collect_date_time(forecastDemand, "DATETIME")
temperature = collect_date_time(temperature, "DATETIME")
prices = collect_date_time(prices, "SETTLEMENTDATE")
totalDemand = collect_date_time(totalDemand, "DATETIME")
totalDemand.head()

DATETIME REGIONID  TOTALDEMAND        DATE  YEAR  MONTH  DAY  \
0 2010-01-01 00:00:00     NSW1      7997.41  2010-01-01  2010      1    1   
1 2010-01-01 00:05:00     NSW1      7946.51  2010-01-01  2010      1    1   
2 2010-01-01 00:10:00     NSW1      7921.55  2010-01-01  2010      1    1   
3 2010-01-01 00:15:00     NSW1      7837.84  2010-01-01  2010      1    1   
4 2010-01-01 00:20:00     NSW1      7781.11  2010-01-01  2010      1    1   

       TIME  HOURS  MINUTES  
0  00:00:00      0        0  
1  00:05:00      0        5  
2  00:10:00      0       10  
3  00:15:00      0       15  
4  00:20:00      0       20

In [22]:
def categorise_hours(df, col="HOURS"):
    conditions = [
        (df[col] >= 0) & (df[col] < 3),
        (df[col] >= 3) & (df[col] < 6),
        (df[col] >= 6) & (df[col] < 9),
        (df[col] >= 9) & (df[col] < 12),
        (df[col] >= 12) & (df[col] < 15),
        (df[col] >= 15) & (df[col] < 18),
        (df[col] >= 18) & (df[col] < 21),
        (df[col] >= 21) & (df[col] < 0),
    ]
    choices = [1, 2, 3, 4, 5, 6, 7, 8]
    df["HOURGROUP"] = np.select(conditions, choices, default=0)
    return df

In [23]:
totalDemand = categorise_hours(totalDemand)
forecastDemand = categorise_hours(forecastDemand)
temperature = categorise_hours(temperature)
prices = categorise_hours(prices)
prices.head()

REGION      SETTLEMENTDATE  TOTALDEMAND    RRP PERIODTYPE        DATE  YEAR  \
0   NSW1 2010-01-01 00:30:00      7809.31  21.50      TRADE  2010-01-01  2010   
1   NSW1 2010-01-01 01:00:00      7483.69  20.98      TRADE  2010-01-01  2010   
2   NSW1 2010-01-01 01:30:00      7117.23  20.06      TRADE  2010-01-01  2010   
3   NSW1 2010-01-01 02:00:00      6812.03  17.93      TRADE  2010-01-01  2010   
4   NSW1 2010-01-01 02:30:00      6544.33  17.43      TRADE  2010-01-01  2010   

   MONTH  DAY      TIME  HOURS  MINUTES  HOURGROUP  
0      1    1  00:30:00      0       30          1  
1      1    1  01:00:00      1        0          1  
2      1    1  01:30:00      1       30          1  
3      1    1  02:00:00      2        0          1  
4      1    1  02:30:00      2       30          1

#### Remove redundant columns

In [24]:
totalDemand = totalDemand.drop(columns=["REGIONID","DATETIME","TIME","HOURS","MINUTES","DATE"], errors='ignore')
forecastDemand = forecastDemand.drop(columns=["REGIONID","DATETIME","LASTCHANGED","PREDISPATCHSEQNO","PERIODID","TIME","HOURS","MINUTES","DATE"], errors='ignore')
temperature = temperature.drop(columns=["LOCATION","DATETIME","DATETIME","TIME","HOURS","MINUTES","DATE"], errors='ignore')
prices = prices.drop(columns=["REGION","PERIODTYPE","SETTLEMENTDATE","TIME","HOURS","MINUTES","DATE"], errors='ignore')
bomSolar = bomSolar.drop(columns=["Product code","Bureau of Meteorology station number"], errors='ignore')
bomTemp = bomTemp.drop(columns=["Product code","Bureau of Meteorology station number","Quality"], errors='ignore')
bomRain = bomRain.drop(columns=["Product code","Bureau of Meteorology station number","Quality"], errors='ignore')

#### Merge Datasets

In [25]:
# bom datasets - merge and rename headers
bom = bomTemp.merge(bomRain, how="outer", on=["Year","Month","Day"])
bom = bom.merge(bomSolar, how="outer", on=["Year","Month","Day"])
bom.columns = ['YEAR','MONTH','DAY','MAX','ACCMAX','RAIN','RAINPERIOD','SOLAR']
bom.head(2)

YEAR  MONTH  DAY   MAX  ACCMAX  RAIN  RAINPERIOD  SOLAR
0  2010      1    1  29.6     1.0   0.2         1.0   14.6
1  2010      1    2  29.5     1.0   0.0         1.0   18.4

In [26]:
# aggregate forecastDemand - memory erro 
forecastDemandAgg = forecastDemand.groupby(["YEAR","MONTH","DAY","HOURGROUP"]).mean().reset_index()
forecastDemandAgg.head()

YEAR  MONTH  DAY  HOURGROUP  FORECASTDEMAND
0  2010      1    1          0     7790.240662
1  2010      1    1          1     7199.543706
2  2010      1    1          2     6139.117944
3  2010      1    1          3     6571.026103
4  2010      1    1          4     7967.543253

In [30]:
df = totalDemand.merge(bom, how="left", on=["YEAR","MONTH","DAY"])
df = df.merge(prices, how="left", on=["YEAR","MONTH","DAY","HOURGROUP"])
df = df.merge(forecastDemandAgg, how="left", on=["YEAR","MONTH","DAY","HOURGROUP"])
df

TOTALDEMAND_x  YEAR  MONTH  DAY  HOURGROUP   MAX  ACCMAX  RAIN  \
0               7997.41  2010      1    1          1  29.6     1.0   0.2   
1               7997.41  2010      1    1          1  29.6     1.0   0.2   
2               7997.41  2010      1    1          1  29.6     1.0   0.2   
3               7997.41  2010      1    1          1  29.6     1.0   0.2   
4               7997.41  2010      1    1          1  29.6     1.0   0.2   
5               7997.41  2010      1    1          1  29.6     1.0   0.2   
6               7997.41  2010      1    1          1  29.6     1.0   0.2   
7               7997.41  2010      1    1          1  29.6     1.0   0.2   
8               7997.41  2010      1    1          1  29.6     1.0   0.2   
9               7997.41  2010      1    1          1  29.6     1.0   0.2   
10              7946.51  2010      1    1          1  29.6     1.0   0.2   
11              7946.51  2010      1    1          1  29.6     1.0   0.2   
12              7946.51  2010      1    1          1  29.6     1.0   0.2   
13              7946.51  2010      1    1          1  29.6     1.0   0.2   
14              7946.51  2010      1    1          1  29.6     1.0   0.2   
15              7946.51  2010      1    1          1  29.6     1.0   0.2   
16              7946.51  2010      1    1          1  29.6     1.0   0.2   
17              7946.51  2010      1    1          1  29.6     1.0   0.2   
18              7946.51  2010      1    1          1  29.6     1.0   0.2   
19              7946.51  2010      1    1          1  29.6     1.0   0.2   
20              7921.55  2010      1    1          1  29.6     1.0   0.2   
21              7921.55  2010      1    1          1  29.6     1.0   0.2   
22              7921.55  2010      1    1          1  29.6     1.0   0.2   
23              7921.55  2010      1    1          1  29.6     1.0   0.2   
24              7921.55  2010      1    1          1  29.6     1.0   0.2   
25              7921.55  2010      1    1          1  29.6     1.0   0.2   
26              7921.55  2010      1    1          1  29.6     1.0   0.2   
27              7921.55  2010      1    1          1  29.6     1.0   0.2   
28              7921.55  2010      1    1          1  29.6     1.0   0.2   
29              7921.55  2010      1    1          1  29.6     1.0   0.2   
...                 ...   ...    ...  ...        ...   ...     ...   ...   
11828436        8027.28  2022      8    1          1  20.4     1.0   0.0   
11828437        8027.28  2022      8    1          1  20.4     1.0   0.0   
11828438        8027.28  2022      8    1          1  20.4     1.0   0.0   
11828439        8027.28  2022      8    1          1  20.4     1.0   0.0   
11828440        8027.28  2022      8    1          1  20.4     1.0   0.0   
11828441        8027.28  2022      8    1          1  20.4     1.0   0.0   
11828442        8027.28  2022      8    1          1  20.4     1.0   0.0   
11828443        8027.28  2022      8    1          1  20.4     1.0   0.0   
11828444        8027.28  2022      8    1          1  20.4     1.0   0.0   
11828445        8027.28  2022      8    1          1  20.4     1.0   0.0   
11828446        8027.28  2022      8    1          1  20.4     1.0   0.0   
11828447        8027.28  2022      8    1          1  20.4     1.0   0.0   
11828448        8027.28  2022      8    1          1  20.4     1.0   0.0   
11828449        8027.28  2022      8    1          1  20.4     1.0   0.0   
11828450        8027.28  2022      8    1          1  20.4     1.0   0.0   
11828451        8027.28  2022      8    1          1  20.4     1.0   0.0   
11828452        8027.28  2022      8    1          1  20.4     1.0   0.0   
11828453        8027.28  2022      8    1          1  20.4     1.0   0.0   
11828454        8027.28  2022      8    1          1  20.4     1.0   0.0   
11828455        8027.28  2022      8    1          1  20.4     1.0   0.0   
11828456        8027.28  2022      8    1          1  20.4